# Bellabeat / Fitbit Case Study (BigQuery Notebook Version)

  1. Standardize and clean the Fitbit tracking tables (daily, hourly, sleep).
  2. Create analysis views that answer the Bellabeat case study questions.
  3. Let a reviewer run the SQL directly in BigQuery.

Data source: public Fitbit Fitness Tracker Data (Kaggle, 30 users, 2016).

*NOTE: the raw tables use MixedCase column names; we will project them
into clean, analysis-friendly views (snake_case, correct types).*



In [ ]:
# sql_engine: bigquery
# output_variable: define_project
# start _sql
_sql = """
PROJECT = "bellabeat-fitness-analysis"
DATASET = "fitbit_tracking_data"
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
define_project = _bqsqlcell.run(_sql)
define_project

### **1. STANDARDIZE DAILY ACTIVITY (source columns are already typed well)**

Source tables (given):
daily_activity_a_to_m
daily_activity_m_to_a

Both have:

Id (INTEGER), ActivityDate (DATE), ... Calories (INTEGER)

In [2]:
# sql_engine: bigquery
# output_variable: standardize_daily_activity
# start _sql
_sql = """
# 1) Canonical activity, calories, steps, intensities (hourly)
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_a_to_m` AS
SELECT
  Id AS id,
  ActivityDate AS activity_date,
  TotalSteps AS total_steps,
  TotalDistance AS total_distance,
  TrackerDistance AS tracker_distance,
  LoggedActivitiesDistance AS logged_activities_distance,
  VeryActiveDistance AS very_active_distance,
  ModeratelyActiveDistance AS moderately_active_distance,
  LightActiveDistance AS light_active_distance,
  SedentaryActiveDistance AS sedentary_active_distance,
  VeryActiveMinutes AS very_active_minutes,
  FairlyActiveMinutes AS fairly_active_minutes,
  LightlyActiveMinutes AS lightly_active_minutes,
  SedentaryMinutes AS sedentary_minutes,
  Calories AS calories
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.daily_activity_a_to_m`;

CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_m_to_a` AS
SELECT
  Id AS id,
  ActivityDate AS activity_date,
  TotalSteps AS total_steps,
  TotalDistance AS total_distance,
  TrackerDistance AS tracker_distance,
  LoggedActivitiesDistance AS logged_activities_distance,
  VeryActiveDistance AS very_active_distance,
  ModeratelyActiveDistance AS moderately_active_distance,
  LightActiveDistance AS light_active_distance,
  SedentaryActiveDistance AS sedentary_active_distance,
  VeryActiveMinutes AS very_active_minutes,
  FairlyActiveMinutes AS fairly_active_minutes,
  LightlyActiveMinutes AS lightly_active_minutes,
  SedentaryMinutes AS sedentary_minutes,
  Calories AS calories
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.daily_activity_m_to_a`;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
standardize_daily_activity = _bqsqlcell.run(_sql)
standardize_daily_activity

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 2. STANDARDIZE HOURLY TABLES
Problem: hourly tables store Id as STRING and ActivityHour as STRING

We need:

id: INT64

activity_hour: TIMESTAMP

average_intensity: FLOAT64 (source is STRING)

In [3]:
# sql_engine: bigquery
# output_variable: standardize_hourly_tables
# start _sql
_sql = """
-- HOURLY CALORIES A→M
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_a_to_m` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  Calories AS calories
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_calories_a_to_m`;

-- HOURLY CALORIES M→A
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_m_to_a` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  Calories AS calories
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_calories_m_to_a`;

-- HOURLY INTENSITIES A→M
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_a_to_m` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  TotalIntensity AS total_intensity,
  SAFE_CAST(AverageIntensity AS FLOAT64) AS average_intensity
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_intensities_a_to_m`;

-- HOURLY INTENSITIES M→A
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_m_to_a` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  TotalIntensity AS total_intensity,
  SAFE_CAST(AverageIntensity AS FLOAT64) AS average_intensity
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_intensities_m_to_a`;

-- HOURLY STEPS A→M
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_a_to_m` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  StepTotal AS step_total
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_steps_a_to_m`;

-- HOURLY STEPS M→A
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_m_to_a` AS
SELECT
  SAFE_CAST(Id AS INT64) AS id,
  PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', ActivityHour) AS activity_hour,
  StepTotal AS step_total
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.hourly_steps_m_to_a`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
standardize_hourly_tables = _bqsqlcell.run(_sql)
standardize_hourly_tables

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 3. MERGE A→M AND M→A TABLES (R: bind_rows + distinct)

In [4]:
# sql_engine: bigquery
# output_variable: merge_hourly_tables
# start _sql
_sql = """
-- DAILY (stack both)
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all` AS
SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_a_to_m`
UNION ALL
SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_m_to_a`;

-- HOURLY CALORIES (de-duplicate on id + activity_hour)
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_all` AS
WITH unioned AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_a_to_m`
  UNION ALL
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_m_to_a`
)
SELECT *
FROM unioned
QUALIFY ROW_NUMBER() OVER (PARTITION BY id, activity_hour ORDER BY activity_hour) = 1;

-- HOURLY INTENSITIES
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_all` AS
WITH unioned AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_a_to_m`
  UNION ALL
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_m_to_a`
)
SELECT *
FROM unioned
QUALIFY ROW_NUMBER() OVER (PARTITION BY id, activity_hour ORDER BY activity_hour) = 1;

-- HOURLY STEPS
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_all` AS
WITH unioned AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_a_to_m`
  UNION ALL
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_m_to_a`
)
SELECT *
FROM unioned
QUALIFY ROW_NUMBER() OVER (PARTITION BY id, activity_hour ORDER BY activity_hour) = 1;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
merge_hourly_tables = _bqsqlcell.run(_sql)
merge_hourly_tables

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 4. JOIN HOURLY TABLES TOGETHER (R used full_join by id + activity_hour)

In [5]:
# sql_engine: bigquery
# output_variable: join_hourly_tables
# start _sql
_sql = """
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data` AS
WITH c AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_calories_all`
),
i AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_intensities_all`
),
s AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_steps_all`
)
SELECT
  COALESCE(c.id, i.id, s.id) AS id,
  COALESCE(c.activity_hour, i.activity_hour, s.activity_hour) AS activity_hour,
  c.calories,
  i.total_intensity,
  i.average_intensity,
  s.step_total
FROM c
FULL OUTER JOIN i
  ON c.id = i.id AND c.activity_hour = i.activity_hour
FULL OUTER JOIN s
  ON COALESCE(c.id, i.id) = s.id
 AND COALESCE(c.activity_hour, i.activity_hour) = s.activity_hour;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
join_hourly_tables = _bqsqlcell.run(_sql)
join_hourly_tables

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 5. ENRICH HOURLY DATA WITH DATEPARTS (hour, weekday label, date)

In [6]:
# sql_engine: bigquery
# output_variable: enrich_hourly_data
# start _sql
_sql = """
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data_enriched` AS
SELECT
  id,
  activity_hour,
  EXTRACT(HOUR FROM activity_hour) AS hour,
  CASE EXTRACT(DAYOFWEEK FROM activity_hour)
    WHEN 1 THEN 'Sun'
    WHEN 2 THEN 'Mon'
    WHEN 3 THEN 'Tue'
    WHEN 4 THEN 'Wed'
    WHEN 5 THEN 'Thu'
    WHEN 6 THEN 'Fri'
    WHEN 7 THEN 'Sat'
  END AS day,
  DATE(activity_hour) AS date,
  calories,
  total_intensity,
  average_intensity,
  step_total
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
enrich_hourly_data = _bqsqlcell.run(_sql)
enrich_hourly_data

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 6. SLEEP DATA
Source:

minute_sleep_a_to_m: Id(string), date(string), value(string), logId(string)

minute_sleep_m_to_a: same

We must:

cast Id → INT64,
parse date → TIMESTAMP,
cast value → INT64 and cap at 1 like R

In [7]:
# sql_engine: bigquery
# output_variable: sleep_data
# start _sql
_sql = """
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_minute_all` AS
WITH sleep_am AS (
  SELECT
    SAFE_CAST(Id AS INT64) AS id,
    PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', date) AS sleep_ts,
    IF(SAFE_CAST(value AS INT64) > 1, 1, SAFE_CAST(value AS INT64)) AS value
  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.minute_sleep_a_to_m`
),
sleep_ma AS (
  SELECT
    SAFE_CAST(Id AS INT64) AS id,
    PARSE_TIMESTAMP('%m/%d/%Y %I:%M:%S %p', date) AS sleep_ts,
    IF(SAFE_CAST(value AS INT64) > 1, 1, SAFE_CAST(value AS INT64)) AS value
  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.minute_sleep_m_to_a`
)
SELECT * FROM sleep_am
UNION ALL
SELECT * FROM sleep_ma;

-- Aggregate to daily sleep per user
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_daily` AS
SELECT
  id,
  DATE(sleep_ts) AS sleep_date,
  SUM(value) AS total_minutes_asleep
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_minute_all`
GROUP BY id, sleep_date;

-- Count nights per user
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_night_counts` AS
SELECT
  id,
  COUNT(*) AS n_nights
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_daily`
GROUP BY id;

-- Keep users with at least 10 nights
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_daily_filtered` AS
SELECT d.*
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_daily` d
JOIN `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_night_counts` c
  USING (id)
WHERE c.n_nights >= 10;

-- Average sleep per user, filter out users < 2 hours avg
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_avg_per_user_filtered` AS
SELECT
  id,
  AVG(total_minutes_asleep) AS avg_minutes_asleep,
  COUNT(*) AS n_nights,
  ROUND(AVG(total_minutes_asleep) / 60, 2) AS avg_hours_asleep
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_daily_filtered`
GROUP BY id
HAVING (AVG(total_minutes_asleep) / 60) >= 2;

-- Overall across cleaned users
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_overall_filtered` AS
SELECT
  AVG(avg_hours_asleep) AS overall_avg_hours_asleep,
  AVG(avg_hours_asleep * 60) AS overall_avg_minutes_asleep
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_avg_per_user_filtered`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
sleep_data = _bqsqlcell.run(_sql)
sleep_data

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### 7. ANALYSIS VIEWS (Q1–Q7)


In [8]:
# sql_engine: bigquery
# output_variable: q_one_hourly_averages
# start _sql
_sql = """
-- Q1: Create the analysis view
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q1_hourly_averages` AS
SELECT
  hour,
  AVG(step_total) AS avg_steps,
  AVG(calories) AS avg_calories,
  AVG(average_intensity) AS avg_intensity
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data_enriched`
GROUP BY hour
ORDER BY hour;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_one_hourly_averages = _bqsqlcell.run(_sql)
q_one_hourly_averages

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [9]:
# sql_engine: bigquery
# output_variable: q_one_display
# start _sql
_sql = """
-- Display the results
SELECT hour, avg_steps FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q1_hourly_averages`
ORDER BY hour;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_one_display = _bqsqlcell.run(_sql)
q_one_display

TableWidget(page_size=10, row_count=24, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

In [10]:
# dataframe: q_one_display
# uuid: C193FADF-AF6B-41CF-BC04-9B28ADDEE503
# output_variable:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='q_one_display', uuid='C193FADF-AF6B-41CF-BC04-9B28ADDEE503')

<IPython.core.display.Javascript object>

### Q2: weekday vs weekend

In [ ]:
# sql_engine: bigquery
# output_variable: q_two_weekday_vs_weekend
# start _sql
_sql = """
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q2_weekday_vs_weekend` AS
WITH base AS (
  SELECT
    *,
    CASE
      WHEN day IN ('Sat', 'Sun') THEN 'Weekend'
      ELSE 'Weekday'
    END AS week_part
  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data_enriched`
)
SELECT
  week_part,
  hour,
  AVG(step_total) AS avg_steps
FROM base
GROUP BY week_part, hour
ORDER BY week_part, hour;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_two_weekday_vs_weekend = _bqsqlcell.run(_sql)
q_two_weekday_vs_weekend

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: q_two_display
# start _sql
_sql = """
SELECT week_part, hour, avg_steps
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q2_weekday_vs_weekend`
ORDER BY week_part, hour;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_two_display = _bqsqlcell.run(_sql)
q_two_display

TableWidget(page_size=10, row_count=48, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

In [1]:
# dataframe: q_two_display
# uuid: 1F2A0F65-504C-4B5F-89E5-51D5992A982D
# output_variable:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='q_two_display', uuid='1F2A0F65-504C-4B5F-89E5-51D5992A982D')

KeyError: 'q_two_display'

### Q3 – Relationship between intensity and calories

In [ ]:
# sql_engine: bigquery
# output_variable: q_three_intensity_vs_calories_cor
# start _sql
_sql = """
-- Q3: correlation between total intensity and calories
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q3_corr_calories_intensity` AS
SELECT
  CORR(total_intensity, calories) AS corr_calories_intensity
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data_enriched`;

-- To display it:
SELECT *
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q3_corr_calories_intensity`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_three_intensity_vs_calories_cor = _bqsqlcell.run(_sql)
q_three_intensity_vs_calories_cor

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### Q4 – Steps vs Average Intensity

In [ ]:
# sql_engine: bigquery
# output_variable: q_four_steps_vs_avg_intensity
# start _sql
_sql = """
-- Q4: correlation between steps and average intensity
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q4_corr_steps_avg_intensity` AS
SELECT
  CORR(step_total, average_intensity) AS corr_steps_avg_intensity
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_hourly_data_enriched`;

-- To display it:
SELECT *
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q4_corr_steps_avg_intensity`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_four_steps_vs_avg_intensity = _bqsqlcell.run(_sql)
q_four_steps_vs_avg_intensity

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### Q5 – Average Activity Level (steps) and % meeting goal

In [ ]:
# sql_engine: bigquery
# output_variable: q_five_a
# start _sql
_sql = """
-- Q5a: mean and median steps
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q5_steps_summary` AS
SELECT
  AVG(total_steps) AS mean_steps,
  APPROX_QUANTILES(total_steps, 2)[OFFSET(1)] AS median_steps
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all`;

-- display
SELECT *
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q5_steps_summary`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_five_a = _bqsqlcell.run(_sql)
q_five_a

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: q_five_b
# start _sql
_sql = """
-- Q5b: % of days with >= 10,000 steps
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q5_percent_meeting_goal` AS
WITH labeled AS (
  SELECT
    *,
    total_steps >= 10000 AS met_goal
  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all`
)
SELECT
  100 * AVG(CASE WHEN met_goal THEN 1 ELSE 0 END) AS percent_met_goal
FROM labeled;

-- display
SELECT *
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q5_percent_meeting_goal`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_five_b = _bqsqlcell.run(_sql)
q_five_b

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### Q6 – Sedentary vs Active (sleep-adjusted)

In [ ]:
# sql_engine: bigquery
# output_variable: q_six_a
# start _sql
_sql = """
-- Q6a (revised): activity composition as ratios of waking time
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q6_activity_balance` AS
WITH params AS (
  -- get the actual avg sleep minutes you computed earlier
  SELECT
    overall_avg_minutes_asleep AS avg_sleep_minutes
  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_sleep_overall_filtered`
),
daily AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all`
),
sleep_adjusted AS (
  SELECT
    d.id,
    d.activity_date,
    -- remove sleep time from sedentary, but don't go below 0
    GREATEST(d.sedentary_minutes - p.avg_sleep_minutes, 0) AS sedentary_awake_minutes,
    d.lightly_active_minutes,
    d.fairly_active_minutes,
    d.very_active_minutes
  FROM daily d
  CROSS JOIN params p
),
with_totals AS (
  SELECT
    *,
    (
      sedentary_awake_minutes
      + lightly_active_minutes
      + fairly_active_minutes
      + very_active_minutes
    ) AS waking_minutes_total
  FROM sleep_adjusted
),
with_ratios AS (
  SELECT
    *,
    SAFE_DIVIDE(sedentary_awake_minutes, waking_minutes_total) AS sedentary_ratio,
    SAFE_DIVIDE(lightly_active_minutes,  waking_minutes_total) AS lightly_ratio,
    SAFE_DIVIDE(fairly_active_minutes,   waking_minutes_total) AS fairly_ratio,
    SAFE_DIVIDE(very_active_minutes,     waking_minutes_total) AS very_ratio
  FROM with_totals
)
-- final rollup: average the per-day ratios across all users/days
SELECT
  AVG(sedentary_ratio) AS avg_sedentary_ratio,
  AVG(lightly_ratio)   AS avg_lightly_ratio,
  AVG(fairly_ratio)    AS avg_fairly_ratio,
  AVG(very_ratio)      AS avg_very_ratio
FROM with_ratios;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_six_a = _bqsqlcell.run(_sql)
q_six_a

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: q_six_a
# start _sql
_sql = """
SELECT 'Sedentary (Awake)' AS activity_type, avg_sedentary_ratio AS ratio FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q6_activity_balance`
UNION ALL
SELECT 'Lightly Active',   avg_lightly_ratio  FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q6_activity_balance`
UNION ALL
SELECT 'Fairly Active',    avg_fairly_ratio   FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q6_activity_balance`
UNION ALL
SELECT 'Very Active',      avg_very_ratio     FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q6_activity_balance`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_six_a = _bqsqlcell.run(_sql)
q_six_a

TableWidget(page_size=10, row_count=4, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# dataframe: q_six_a
# uuid: 4F7DB2F1-6171-4CB6-A683-D671317187B8
# output_variable:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='q_six_a', uuid='4F7DB2F1-6171-4CB6-A683-D671317187B8')

<IPython.core.display.Javascript object>

### Q7 – Activity distance vs calories

In [ ]:
# sql_engine: bigquery
# output_variable: q_seven_activity_distance_vs_calories
# start _sql
_sql = """
-- Q7: correlation between total active distance and calories
CREATE OR REPLACE VIEW
  `bellabeat-fitness-analysis.fitbit_tracking_data.v_q7_active_distance_corr` AS
WITH daily AS (
  SELECT * FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all`
),
with_distance AS (
  SELECT
    *,
    COALESCE(very_active_distance, 0)
    + COALESCE(moderately_active_distance, 0)
    + COALESCE(light_active_distance, 0) AS total_active_distance
  FROM daily
)
SELECT
  CORR(total_active_distance, calories) AS corr_active_distance_calories
FROM with_distance;

-- display
SELECT *
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_q7_active_distance_corr`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_seven_activity_distance_vs_calories = _bqsqlcell.run(_sql)
q_seven_activity_distance_vs_calories

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: q_seven_display
# start _sql
_sql = """
SELECT
  COALESCE(very_active_distance, 0)
  + COALESCE(moderately_active_distance, 0)
  + COALESCE(light_active_distance, 0) AS total_active_distance,
  calories
FROM `bellabeat-fitness-analysis.fitbit_tracking_data.v_daily_activity_all`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
q_seven_display = _bqsqlcell.run(_sql)
q_seven_display

TableWidget(page_size=10, row_count=1397, table_html='<table border="1" class="dataframe table table-striped t…

Computation deferred. Computation will process 44.7 kB

In [ ]:
# dataframe: q_seven_display
# uuid: BFE0619C-2E93-4710-A52B-404633A00586
# output_variable:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='q_seven_display', uuid='BFE0619C-2E93-4710-A52B-404633A00586')

<IPython.core.display.Javascript object>

END OF SCRIPT

This single script: standardizes raw Fitbit CSVs, merges A→M and M→A, recreates all hourly joins,rebuilds the sleep analysis, reproduces Q1–Q7 analysis, exposes everything as views for BI or stakeholder decks.